<a href="https://colab.research.google.com/github/Ammon1/Math/blob/master/noise_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
#from keras.layers import 
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Bidirectional


import xgboost as xgb

In [19]:
def import_files(sample,ac_big,ac_small,number):#H67, 100AC,1AC,3
  i=0
  j=0
  for file in arr:
      if 'dpt' in file:
        #pomair dokladny
        if ac_big in file and sample in file:
            globals()['df_Y']=pd.read_csv(path+file,sep=',')
  for file in arr:
        #pomiar niedokladny
        if ac_small in file and sample in file:    
            print('i ',i)
            globals()['df'+str(i)]=pd.read_csv(path+file,sep=',')
            #intensity correction
            globals()['df'+str(i)]=globals()['df'+str(i)]*np.mean(df_Y.iloc[25000:45000,1].values)/np.mean((globals()['df'+str(i)].iloc[25000:45000,1].values))
            #if i==0:
            #  X_0=globals()['df'+str(i)]
            #else:
            # X_0=np.hstack((X_0,globals()['df'+str(i)]))
            i=i+1
  Y_0=df_Y.iloc[:,1].values
  Y_0=Y_0.reshape(-1,1)
  scalerY = StandardScaler()
  Y_0=scalerY.fit_transform(Y_0)
  Y_0=Y_0.flatten()
  
  X_0=np.hstack((
              df0.iloc[:,1],
              df1.iloc[:,1],
              df2.iloc[:,1],
              df3.iloc[:,1],
              df4.iloc[:,1],
              ))
  scalerX = StandardScaler()
  X_0=X_0.reshape(-1,1)
  X_0=scalerX.fit_transform(X_0)
  X_0=X_0.flatten()

  print(X_0.shape)
  print(X_1.shape)
  globals()['Y'+number]=np.tile(Y_0,int(X_0.shape[0]/Y_0.shape[0]))
  globals()['X'+number]=np.vstack((X_0,X_1,X_2,X_3,X_4,X_5,X_8,X_9,X_10,X_11,X_12))
  

  

  plt.figure(figsize=(20,20))
  plt.plot(globals()['X'+number][0,:])
  plt.plot(globals()['Y'+number])
  return globals()['X'+number],globals()['Y'+number]

In [20]:
def create_dataset(X,Y, look_back):
	dataX, dataY = [], []
	for i in range(len(Y)-look_back-1):
		a = X[i:(i+look_back), :]
		dataX.append(a)
		dataY.append(Y[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [21]:
path='/content/drive/MyDrive/fizyka/ftir_noise/res12/'
arr = os.listdir(path)

In [22]:
i=0
j=0
for file in arr:
    if 'dpt' in file:
       #pomair dokladny
       if '500AC' in file and 'MIR' in file:
           print('big')
           globals()['df_Y']=pd.read_csv(path+file,sep=',')
       #pomiar niedokladny
       elif '1AC' in file and 'MIR' in file:    
           print('i ',i)
           globals()['df'+str(i)]=pd.read_csv(path+file,sep=',')
           i=i+1
      #pomiar tla
       elif '1AC' in file and 'bcg' in file:  
           print('bck j ',j)
           globals()['df_bg'+str(j)]=pd.read_csv(path+file,sep=',')
           j=j+1

big
i  0
i  1
i  2
i  3
i  4
bck j  0
bck j  1
bck j  2
bck j  3
bck j  4
bck j  5
bck j  6
bck j  7
bck j  8
bck j  9


In [23]:
def moving_average(a, n) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def moving_average_corr(a, n) :
    ret=moving_average(a, n)
    return np.insert(ret,0,a[:n-1])

In [24]:
Y_0=df_Y.iloc[:,1].values
Y_0=Y_0.reshape(-1,1)
scalerY = StandardScaler()
Y_0=scalerY.fit_transform(Y_0)
Y_0=Y_0.flatten()

X_0=np.hstack((
            df1.iloc[:,1],
            df2.iloc[:,1],
            df3.iloc[:,1],
            df4.iloc[:,1],
            df0.iloc[:,1],))

scalerX = StandardScaler()
X_0=X_0.reshape(-1,1)
X_0=scalerX.fit_transform(X_0)
X_0=X_0.flatten()

X_01=np.arange(0,df_Y.iloc[:,1].shape[0])
X_02=np.power(X_01,2)
X_03=np.power(X_01,3)

X0=df_bg0.iloc[:,1].values
X1=df_bg1.iloc[:,1].values
X2=df_bg2.iloc[:,1].values
X3=df_bg3.iloc[:,1].values
X4=df_bg4.iloc[:,1].values
X5=df_bg5.iloc[:,1].values
X6=df_bg6.iloc[:,1].values
X7=df_bg7.iloc[:,1].values
X8=df_bg8.iloc[:,1].values
X9=df_bg9.iloc[:,1].values


X_1=np.tile(X1,int(X_0.shape[0]/Y_0.shape[0]))
X_2=np.tile(X2,int(X_0.shape[0]/Y_0.shape[0]))
X_3=np.tile(X3,int(X_0.shape[0]/Y_0.shape[0]))
X_4=np.tile(X4,int(X_0.shape[0]/Y_0.shape[0]))
X_5=np.tile(X5,int(X_0.shape[0]/Y_0.shape[0]))
X_6=np.tile(X6,int(X_0.shape[0]/Y_0.shape[0]))
X_7=np.tile(X7,int(X_0.shape[0]/Y_0.shape[0]))
X_8=np.tile(X8,int(X_0.shape[0]/Y_0.shape[0]))
X_9=np.tile(X9,int(X_0.shape[0]/Y_0.shape[0]))

X_10=np.tile(X_01,int(X_0.shape[0]/Y_0.shape[0]))
X_11=np.tile(X_02,int(X_0.shape[0]/Y_0.shape[0]))
X_12=np.tile(X_03,int(X_0.shape[0]/Y_0.shape[0]))
X_13=np.tile(X_04,int(X_0.shape[0]/Y_0.shape[0]))

Y=np.tile(Y_0,int(X_0.shape[0]/Y_0.shape[0]))
X=np.vstack((X_0,X_1,X_2,X_3,X_4,X_5,X_8,X_9,X_10,X_11,X_12))

NameError: ignored

In [ ]:
print(X_0.shape)
print(X_1.shape)

In [ ]:
X1,Y1=import_files('1668','500AC','1AC','1')

In [ ]:
X2,Y2=import_files('testowa','500AC','1AC','2')

In [ ]:
X3,Y3=import_files('10301','500AC','1AC','2')

In [ ]:
X4,Y4=import_files('pudelko1','500AC','1AC','2')

In [ ]:
X5,Y5=import_files('pudelko2','500AC','2AC','5')

In [ ]:
print(X.shape)
print(X1.shape)
print(X2.shape)
print(X3.shape)
print(X4.shape)
print(X5.shape)
print(Y.shape)

In [ ]:
X=np.transpose(X)
X1=np.transpose(X1)
X2=np.transpose(X2)
X3=np.transpose(X3)
X4=np.transpose(X4)
X5=np.transpose(X5)

In [ ]:
X_fin=np.vstack((X,X1,X2,X3,X4,X5))
Y_fin=np.hstack((Y,Y1,Y2,Y3,Y4,Y5))

In [ ]:
plt.plot(X[:,0])
plt.plot(Y)

In [ ]:
plt.plot(X_fin[:,0])
plt.plot(Y_fin)

In [ ]:
scalerX = StandardScaler()
scalerY = StandardScaler()
X_fin=scalerX.fit_transform(X_fin)
Y_fin=Y_fin.reshape(-1,1)
Y_fin=scalerY.fit_transform(Y_fin)

In [ ]:
print(X_fin.shape[0]/Y_0.shape[0])
print(Y_fin.shape)

In [ ]:
X_fin=np.round(X_fin,decimals=6)
Y_fin=np.round(Y_fin,decimals=6)

In [ ]:
plt.plot(X_fin[:,0])
plt.plot(Y_fin)

In [ ]:
X_train,y_train=X_fin[:25*Y_0.shape[0],:],Y_fin[:25*Y_0.shape[0]]
X_test,y_test=X_fin[25*Y_0.shape[0]:26*Y_0.shape[0],:],Y_fin[25*Y_0.shape[0]:26*Y_0.shape[0]]

In [ ]:
plt.plot(X_train[24*Y_0.shape[0]:,0])
plt.plot(y_train[24*Y_0.shape[0]:])

In [ ]:
plt.plot(y_train[24*Y_0.shape[0]:])
plt.plot(y_test[:])

In [ ]:
look_back =30
trainX, trainY = create_dataset(X_train,y_train, look_back)
testX, testY = create_dataset(X_test,y_test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 11, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 11, testX.shape[1]))

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(16, input_shape=(12, look_back),kernel_initializer='he_uniform')))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop',metrics=['mse','mae'])
history=model.fit(trainX, trainY,validation_data=(testX, testY), epochs=10, batch_size=10000)

plt.plot(history.history['mse'])
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')
plt.ylim(0.0001,0.1)
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
model.fit(trainX, trainY,validation_data=(testX, testY), epochs=1, batch_size=10000)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(32, input_shape=(11, look_back),kernel_initializer='he_uniform')))
model.add(Dense(64, activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop',metrics=['mse','mae'])
history=model.fit(trainX, trainY,validation_data=(testX, testY), epochs=10, batch_size=10000)

plt.plot(history.history['mse'])
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')

plt.legend()
plt.show()

In [ ]:
model.fit(trainX, trainY,validation_data=(testX, testY), epochs=1, batch_size=10000)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
plt.xlim(25000,40000)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(32, input_shape=(11, look_back),kernel_initializer='he_uniform')))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop',metrics=['mse','mae'])
history=model.fit(trainX, trainY,validation_data=(testX, testY), epochs=10, batch_size=10000)

plt.plot(history.history['mse'])
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')

plt.legend()
plt.show()

In [ ]:
model.fit(trainX, trainY,validation_data=(testX, testY), epochs=1, batch_size=10000)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')
#plt.xlim(25000,35000)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(16, input_shape=(11, look_back),kernel_initializer='he_uniform')))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop',metrics=['mse','mae'])
history=model.fit(trainX, trainY,validation_data=(testX, testY), epochs=10, batch_size=10000)

plt.plot(history.history['mse'])
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')

plt.legend()
plt.show()

In [ ]:
model.fit(trainX, trainY,validation_data=(testX, testY), epochs=1, batch_size=10000)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(16, input_shape=(11, look_back),kernel_initializer='he_uniform')))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop',metrics=['mse','mae'])
history=model.fit(trainX, trainY,validation_data=(testX, testY), epochs=5, batch_size=10000)

plt.plot(history.history['mse'])
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')

plt.legend()
plt.show()

In [ ]:
model.fit(trainX, trainY,validation_data=(testX, testY), epochs=1, batch_size=10000)
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
plt.figure(figsize=(20,10))
plt.plot(testPredict,color='red')
plt.plot(testY,color='black')
plt.plot(testX[:,0,0],color='green')